## gathering results / vote count numbers captured over time and collecting them into a datastore (db / pqt)
- capture CHANGES in the numbers
- if subsequent entries for the same item show no change in number, then skip it

### what to do about initial data where st code, S/U wasn't propery recorded:
- the pc_name + pc_num might be unique for the entire dataset. If so, lookup and get the missing stuff and record it
- if not possible to disambiguate, skip it## what to do about initial data where st code, S/U wasn't propery recorded:
- the pc_name + pc_num might be unique for the entire dataset. If so, lookup and get the missing stuff and record it
- if not possible to disambiguate, skip it

In [1]:
import pandas as pd
import os

In [2]:
pcfile = '2024-PC-attributes.csv'
folder1 = 'state_wise_results'
folder2 = 'pc_wise_results'

In [3]:
df1 = pd.read_csv(pcfile, dtype=str).fillna('')

In [4]:
df1.sample(5)

,OBJECTID,pc_id,s_u,st_code,st_name,pc_no,pc_name,pc_name_alias,status,State_LGD,...,Phase,DOP,Poll_Day,DOA,DOIN,LDFN,DSN,LDWC,DOCV,Date_of_Poll
110,144,601,S,7,HARYANA,1,Ambala,Ambala,,6,...,6TH PHASE,,Saturday,16-3-2024(Saturday),29-4-2024(Monday),6-5-2024(Monday),07-5-2024(Tuesday),09-5-2024(Thursday),4-6-2024(Tuesday),5/25/2024
266,268,2112,S,18,ODISHA,12,Nabarangpur,Nabarangpur,,21,...,4TH PHASE,,Monday,16-3-2024(Saturday),18-4-2024(Thursday),25-4-2024(Thursday),26-4-2024(Friday),29-4-2024(Monday),4-6-2024(Tuesday),5/13/2024
111,145,602,S,7,HARYANA,2,Kurukshetra,Kurukshetra,,6,...,6TH PHASE,,Saturday,16-3-2024(Saturday),29-4-2024(Monday),6-5-2024(Monday),07-5-2024(Tuesday),09-5-2024(Thursday),4-6-2024(Tuesday),5/25/2024
144,521,2921,S,10,KARNATAKA,21,Mysore,Mysore,,29,...,2ND PHASE,,Friday,16-3-2024(Saturday),28-3-2024(Thursday),4-4-2024(Thursday),05-4-2024(Friday),08-4-2024(Monday,4-6-2024(Tuesday),4/26/2024
77,78,1036,S,4,BIHAR,36,Jahanabad,Jahanabad,,10,...,7TH PHASE,,Saturday,16-3-2024(Saturday),07-5-2024(Tuesday),14-5-2024(Tuesday),15-5-2024(Wednesday),17-5-2024(Friday),4-6-2024(Tuesday),6/1/2024


In [5]:
state_files = sorted(os.listdir(folder1)) # always sort!
state_files[:2]

['results_2024-06-04 13:52:24.csv', 'results_2024-06-04 13:54:13.csv']

In [6]:
collector = []
checkMatch = False
missing_columns_count = 0
for file1 in state_files:
    df2 = pd.read_csv(os.path.join(folder1,file1), dtype=str).fillna('')
    # clean up extra spaces
    df2 = df2.map(lambda x: x.strip() if isinstance(x, str) else x)
    df2['file'] = file1
    missing_columns = [x for x in ('s_u', 'st_code', 'st_name') if x not in df2.columns]
    if len(missing_columns):
        missing_columns_count += 1
        for x in missing_columns:
            df2[x] = ''
        for N,r in df2.iterrows():
            findmatch = df1[(df1['pc_no']==r['pc_no']) & (df1['pc_name_alias'].str.lower()==r['pc_name'].lower())]
            if len(findmatch) == 1:
                foundrow = findmatch.to_dict(orient='records')[0]
                for x in missing_columns:
                    df2.at[N,x] = foundrow[x]
            else:
                # print no-match
                print(f"{r['pc_no']} |{r['pc_name']}| {r['source_url']} {len(findmatch)}")
                checkMatch = True
    
    if checkMatch:
        print(f"stopping at {file1} for name correction, {len(collector)} earlier files collected")
        break
    collector.append(df2)

if not checkMatch:
    print(f"{len(collector)} files data gathered, no mismatches. {missing_columns_count} of {len(state_files)} files were missing state columns that were back-filled by matching pc name & number")

196 files data gathered, no mismatches. 50 of 196 files were missing state columns that were back-filled by matching pc name & number


In [7]:
df3 = pd.concat(collector, ignore_index=True, sort=False).sort_values('timestamp').reset_index(drop=True)
# sorting the combined data by timestamp so that we have earlier entries first

In [8]:
df3

,pc_name,pc_no,leading_candidate,leading_party,trailing_candidate,trailing_party,margin,result_status,source_url,timestamp,file,s_u,st_code,st_name
0,Amalapuram (SC),7,G M HARISH (BALAYOGI),Telugu Desam,RAPAKA VARAPRASADA RAO,Yuvajana Sramika Rythu Congress Party,177805,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 13:47:13,results_2024-06-04 13:52:24.csv,S,1,ANDHRA PRADESH
1,Rajampet,24,P V MIDHUN REDDY,Yuvajana Sramika Rythu Congress Party,NALLARI KIRAN KUMAR REDDY,Bharatiya Janata Party,44104,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 13:47:13,results_2024-06-04 13:52:24.csv,S,1,ANDHRA PRADESH
2,Ongole,16,MAGUNTA SREENIVASULU REDDY,Telugu Desam,DR CHEVIREDDY BHASKAR REDD,Yuvajana Sramika Rythu Congress Party,14778,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 13:47:13,results_2024-06-04 13:52:24.csv,S,1,ANDHRA PRADESH
3,Nellore,22,PRABHAKAR REDDY VEMIREDDY,Telugu Desam,VENUMBAKA VIJAYASAI REDDY,Yuvajana Sramika Rythu Congress Party,126148,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 13:47:13,results_2024-06-04 13:52:24.csv,S,1,ANDHRA PRADESH
4,Narsaraopet,14,LAVU SRIKRISHNA DEVARAYALU,Telugu Desam,ANIL KUMAR POLUBOINA,Yuvajana Sramika Rythu Congress Party,93403,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 13:47:13,results_2024-06-04 13:52:24.csv,S,1,ANDHRA PRADESH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102135,ANANTNAG-RAJOURI,3,MIAN ALTAF AHMAD,Jammu & Kashmir National Conference,MEHBOOBA MUFTI,Jammu & Kashmir Peoples Democratic Party,281794,Result Declared,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 07:37:28,results_2024-06-05 07:37:21.csv,U,8,JAMMU & KASHMIR
102136,BARAMULLA,1,ABDUL RASHID SHEIKH,Independent,OMAR ABDULLAH,Jammu & Kashmir National Conference,204142,Result Declared,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 07:37:28,results_2024-06-05 07:37:21.csv,U,8,JAMMU & KASHMIR
102137,JAMMU,5,JUGAL KISHORE,Bharatiya Janata Party,RAMAN BHALLA,Indian National Congress,135498,Result Declared,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 07:37:28,results_2024-06-05 07:37:21.csv,U,8,JAMMU & KASHMIR
102138,SRINAGAR,2,AGA SYED RUHULLAH MEHDI,Jammu & Kashmir National Conference,WAHEED UR REHMAN PARA,Jammu & Kashmir Peoples Democratic Party,188416,Result Declared,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 07:37:28,results_2024-06-05 07:37:21.csv,U,8,JAMMU & KASHMIR


In [9]:
len(df3)

102140

In [10]:
df3.columns

Index(['pc_name', 'pc_no', 'leading_candidate', 'leading_party',
       'trailing_candidate', 'trailing_party', 'margin', 'result_status',
       'source_url', 'timestamp', 'file', 's_u', 'st_code', 'st_name'],
      dtype='object')

In [11]:
df4 = df3.drop_duplicates(['s_u','st_code','pc_no', 'margin', 'leading_party', 'result_status' ]).copy().sort_values(['s_u','st_code','pc_no','timestamp']).reset_index(drop=True)
len(df4)

13283

In [16]:
print(f"After de-duplicating entries where there were no changes recorded, data reduced from {len(df3)} to {len(df4)} rows")

After de-duplicating entries where there were no changes recorded, data reduced from 102140 to 13283 rows


In [15]:
df4.sample(10)

,pc_name,pc_no,leading_candidate,leading_party,trailing_candidate,trailing_party,margin,result_status,source_url,timestamp,file,s_u,st_code,st_name
2461,Bhiwandi,23,BALYA MAMA - SURESH GOPINATH MHATRE,Nationalist Congress Party – Sharadchandra Pawar,KAPIL MORESHWAR PATIL,Bharatiya Janata Party,27361,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 13:54:37,results_2024-06-04 13:54:47.csv,S,13,MAHARASHTRA
12911,Ahmedabad East,7,HASMUKHBHAI PATEL (H.S.PATEL),Bharatiya Janata Party,HIMMATSINH PRAHLADSINH PATEL,Indian National Congress,451008,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 16:08:13,results_2024-06-04 16:10:45.csv,S,6,GUJARAT
1597,BHOPAL,19,ALOK SHARMA,Bharatiya Janata Party,ADVOCATE ARUN SHRIVASTAVA,Indian National Congress,478422,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 18:09:44,results_2024-06-04 18:09:17.csv,S,12,MADHYA PRADESH
11531,Barpeta,3,PHANI BHUSAN CHOUDHURY,Asom Gana Parishad,DEEP BAYAN,Indian National Congress,213418,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 18:20:22,results_2024-06-04 18:20:21.csv,S,3,ASSAM
5715,NAGAPATTINAM,29,SELVARAJ V,Communist Party of India,DR SURSITH SANKAR G,All India Anna Dravida Munnetra Kazhagam,152903,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 16:36:42,results_2024-06-04 16:36:27.csv,S,22,TAMIL NADU
1590,BHOPAL,19,ALOK SHARMA,Bharatiya Janata Party,ADVOCATE ARUN SHRIVASTAVA,Indian National Congress,380685,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 16:42:53,results_2024-06-04 16:42:02.csv,S,12,MADHYA PRADESH
12415,Jahanabad,36,SURENDRA PRASAD YADAV,Rashtriya Janata Dal,CHANDESHWAR PRASAD,Janata Dal (United),117621,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 17:23:13,results_2024-06-04 17:23:10.csv,S,4,BIHAR
13132,BHIWANI-MAHENDRAGARH,8,DHARAMBIR SINGH,Bharatiya Janata Party,RAO DAN SINGH,Indian National Congress,39237,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 14:04:02,results_2024-06-04 14:04:12.csv,S,7,HARYANA
7595,Banda,48,KRISHNA DEVI SHIVSHANKER PATEL,Samajwadi Party,R. K. SINGH PATEL,Bharatiya Janata Party,71197,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 17:58:33,results_2024-06-04 17:58:05.csv,S,24,UTTAR PRADESH
12641,Kachchh,1,CHAVDA VINOD LAKHAMSHI,Bharatiya Janata Party,NITESH PARBATBHAI LALAN (MATANG),Indian National Congress,241060,Result in Progress,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 13:54:34,results_2024-06-04 13:54:47.csv,S,6,GUJARAT


In [19]:
columns_order = ["s_u", "st_code", "pc_no", "st_name", "pc_name", 
    "timestamp", "margin", "result_status", 
    "leading_candidate", "leading_party", "trailing_candidate", "trailing_party", 
    "source_url", #"file"
]

In [20]:
df4[columns_order].to_csv('Elections2024_leads_changes.csv', index=False)